In [0]:
# Change these two lines
ssh_key = "ssh-rsa AAAAB3N..."
ngrok_token = "sEv..."

import json
import re

# Download OpenSSH and ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip
! apt-get install openssh-server > /dev/null

# Setup OpenSSH
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PubkeyAuthentication    yes" >> /etc/ssh/sshd_config
! echo "AuthorizedKeysFile  .ssh/authorized_keys" >> /etc/ssh/sshd_config

# Setup CUDA
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

# Copy ngrok token and SSH key
! mkdir -p ~/.ssh
! touch ~/.ssh/authorized_keys
! echo $ssh_key > ~/.ssh/authorized_keys
! ./ngrok authtoken $ngrok_token > /dev/null

# Restart SSH server
! /etc/init.d/ssh restart > /dev/null

# Start ngrok tunnel
get_ipython().system_raw("./ngrok tcp 22 &")

# Get SSH address
for i in range(20):
    try:
        ngrok_info = ! curl -s http://localhost:4040/api/tunnels
        ngrok_info = str("".join(ngrok_info))
        url = json.loads(ngrok_info)["tunnels"][0]["public_url"]
        domain = re.search("tcp://(.*):", url).group(1)
        port = re.search(":([0-9]*$)", url).group(1)
        print("ssh root@" + domain + " -p " + port)
    except Exception as e:
        pass
    else:
        break

    print("Unable to open SSH.")